# Yield statistics for algorithms each enterprise


### Configurations

In [43]:
import logging

logging.basicConfig(format='%(asctime)s %(levelname)s %(message)s',

    datefmt='%Y-%m-%d,%H:%M:%S:%f', level=logging.INFO)

In [44]:
logging.info('Application Start!!')

2022-06-11,03:34:07:%f INFO Application Start!!


### Get company

In [45]:
import findspark
findspark.init()

In [46]:
from pyspark import SparkConf, SparkContext, SQLContext


conf = SparkConf().setMaster("yarn").setAppName("bigdata-group1-top-algo")
sc = SparkContext.getOrCreate(conf = conf)
sqlContext = SQLContext(sc)


In [47]:
company_df = sqlContext.read.option("multiline", "true").json("data/companys.json")

In [48]:
company_df.printSchema()

root
 |-- company: string (nullable = true)
 |-- difficulty: string (nullable = true)
 |-- title: string (nullable = true)
 |-- type: array (nullable = true)
 |    |-- element: string (containsNull = true)



In [20]:
company_df.groupBy('difficulty').count().show()

+------------+-----+
|  difficulty|count|
+------------+-----+
|다이아몬드 5|    3|
|  플래티넘 4|    5|
|  플래티넘 1|    4|
|      실버 4|   16|
|  플래티넘 3|    5|
|      실버 2|   18|
|      실버 1|   16|
|    브론즈 2|    7|
|      골드 2|   28|
|  플래티넘 2|    3|
|    브론즈 1|    2|
|      골드 3|   28|
|      골드 1|   14|
|      골드 5|   52|
|다이아몬드 3|    1|
|      골드 4|   46|
|      실버 3|   12|
|      실버 5|   12|
|  플래티넘 5|   16|
+------------+-----+



In [49]:
from pyspark.sql import functions as F
score_map = {"브론즈 5": 1, "브론즈 4": 2, "브론즈 3": 3, "브론즈 2": 4, "브론즈 1": 5,
            "실버 5": 6, "실버 4": 7, "실버 3": 8, "실버 2": 9, "실버 1": 10,
            "골드 5": 11, "골드 4": 12, "골드 3": 13, "골드 2": 14, "골드 1": 15,
            "플래티넘 5": 16, "플래티넘 4": 17, "플래티넘 3": 18, "플래티넘 2": 19, "플래티넘 1": 20,
             "다이아몬드 5": 21, "다이아몬드 4": 22, "다이아몬드 3": 23, "다이아몬드 2": 24, "다이아몬드 1": 25,
            }

In [50]:
score_udf = F.udf(lambda x: score_map[x])
score_df = company_df.withColumn("difficulty_score", score_udf(F.col("difficulty")))

In [51]:
type_df = score_df.withColumn("type", F.explode("type"))

In [32]:
score_df.na.drop(subset=["type"]).withColumn("type", F.explode("type")).count()

845

In [52]:
company_type_g_df = type_df.groupBy("company", "type") \
  .agg(F.count('type').alias("count"), F.avg("difficulty_score").alias("avg_of_difficulty")) \
  .sort("company", F.col("count").desc())
company_type_g_df.persist()

DataFrame[company: string, type: string, count: bigint, avg_of_difficulty: double]

In [53]:
company_type_g_df.show()

+-------+-----------------------+-----+------------------+
|company|                   type|count| avg_of_difficulty|
+-------+-----------------------+-----+------------------+
|coupang|                 문자열|   22|10.363636363636363|
|coupang|                   구현|   20|              10.7|
|coupang|              자료 구조|   12|             12.25|
|coupang|                   수학|   11|              13.0|
|coupang|            그래프 탐색|   10|              15.7|
|coupang|            그래프 이론|   10|              15.7|
|coupang|         깊이 우선 탐색|    8|             15.25|
|coupang|해시를 사용한 집합과 맵|    7| 9.428571428571429|
|coupang|                   파싱|    7|12.714285714285714|
|coupang|    다이나믹 프로그래밍|    7|12.857142857142858|
|coupang|    브루트포스 알고리즘|    7|11.571428571428571|
|coupang|         너비 우선 탐색|    6|              15.0|
|coupang|        그리디 알고리즘|    4|             10.25|
|coupang|                   트리|    4|             16.75|
|coupang|              이분 탐색|    3|              16.0|
|coupang|     

In [41]:
company_type_g_df.write.format("csv").mode('overwrite') \
    .option("encoding", "UTF-8").save("static/stat_enterprise/stat_each_enterprise")

In [42]:
sc.stop()